# 🎨 MangaGen - AI Manga Generation Pipeline

Generate complete manga pages with **consistent characters** from text prompts!

## Features
- 📝 **Gemini 2.0 Flash** - Story to scene planning
- 🎨 **SDXL + IP-Adapter** - High-quality anime art with character consistency
- 💬 **Smart bubble placement** - AI-powered dialogue positioning
- 📄 **PDF output** - Ready to share/print

## Setup
1. **Add Gemini API Key**: Click the 🔑 icon in the left sidebar → Add `GEMINI_API_KEY`
2. **Enable GPU**: Runtime → Change runtime type → T4 GPU
3. **Mount Google Drive**: For model caching (saves time on reruns!)

---

## 🔧 Cell 1: Setup & Dependencies

This cell:
1. Mounts Google Drive for model caching (optional but recommended)
2. Clones the repository
3. Installs dependencies

In [ ]:
#@title 🔧 Setup (Run this first!)
#@markdown Mount Google Drive for model caching?
mount_drive = True  #@param {type:"boolean"}
#@markdown This saves ~10 min on subsequent runs by caching models to your Drive.

import os
import sys

# Mount Google Drive
if mount_drive:
    from google.colab import drive
    drive.mount('/content/drive')
    CACHE_DIR = '/content/drive/MyDrive/MangaGen_Models'
    os.makedirs(CACHE_DIR, exist_ok=True)
    print(f"✅ Drive mounted. Model cache: {CACHE_DIR}")
else:
    CACHE_DIR = '/content/model_cache'
    os.makedirs(CACHE_DIR, exist_ok=True)
    print("⚠️ Drive not mounted. Models will be re-downloaded each session.")

# Set environment variable for cache
os.environ['HF_HOME'] = CACHE_DIR
os.environ['TRANSFORMERS_CACHE'] = CACHE_DIR
os.environ['DIFFUSERS_CACHE'] = CACHE_DIR

# Check GPU
import torch
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_mem = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"✅ GPU: {gpu_name} ({gpu_mem:.1f} GB)")
else:
    print("❌ No GPU detected! Go to Runtime → Change runtime type → T4 GPU")

print("")
print("📦 Cloning repository...")
!rm -rf /content/manga-gen
!git clone --branch mvp/kaggle-flux https://github.com/Barun-2005/manga-gen-ai-pipeline.git /content/manga-gen
%cd /content/manga-gen
print("✅ Repository cloned")

In [ ]:
#@title 📚 Install Dependencies (takes ~2 min first time)

print("📦 Installing dependencies...")
print("   This is much cleaner than Kaggle - no conflicts! 🎉")
print("")

# Install core dependencies
!pip install -q \
    diffusers==0.30.3 \
    transformers>=4.41.0 \
    accelerate>=0.30.0 \
    safetensors>=0.4.0 \
    google-generativeai>=0.5.0 \
    pydantic>=2.0.0 \
    opencv-python-headless>=4.8.0 \
    Pillow>=10.0.0 \
    reportlab>=4.0.0 \
    python-dotenv>=1.0.0 \
    tqdm>=4.65.0 \
    xformers

# Verify
print("")
print("🔍 Verifying installations...")
import diffusers, transformers, torch
print(f"✅ diffusers: {diffusers.__version__}")
print(f"✅ transformers: {transformers.__version__}")
print(f"✅ torch: {torch.__version__}")
print(f"✅ CUDA: {torch.cuda.is_available()}")

# Test SDXL import
from diffusers import StableDiffusionXLPipeline
print("✅ StableDiffusionXLPipeline: can import!")

print("")
print("🎉 All dependencies installed successfully!")

## 🔑 Cell 2: API Key Setup

In [ ]:
#@title 🔑 Set Up Gemini API Key
#@markdown Get your API key from: https://aistudio.google.com/app/apikey

import os

# Try to get from Colab secrets first
try:
    from google.colab import userdata
    os.environ['GEMINI_API_KEY'] = userdata.get('GEMINI_API_KEY')
    print("✅ GEMINI_API_KEY loaded from Colab secrets")
except:
    # Manual input fallback
    if 'GEMINI_API_KEY' not in os.environ or not os.environ['GEMINI_API_KEY']:
        from getpass import getpass
        api_key = getpass("Enter your Gemini API key: ")
        os.environ['GEMINI_API_KEY'] = api_key
        print("✅ API key set manually")
    else:
        print("✅ GEMINI_API_KEY already set")

# Verify
if os.environ.get('GEMINI_API_KEY'):
    print(f"   Key: {os.environ['GEMINI_API_KEY'][:8]}...")
else:
    print("❌ No API key set!")

## ⚙️ Cell 3: Configuration

In [ ]:
#@title ⚙️ Manga Configuration
#@markdown Customize your manga generation:

#@markdown ---
#@markdown ### Story Prompt
STORY_PROMPT = "Astra, a determined space scavenger with messy silver hair and an orange jumpsuit, explores a derelict spaceship and discovers a glowing blue artifact." #@param {type:"string"}

#@markdown ---
#@markdown ### Visual Style
STYLE = "bw_manga" #@param ["bw_manga", "color_anime"]

#@markdown ### Panel Layout
LAYOUT = "2x2" #@param ["2x2", "vertical_webtoon", "3_panel", "single"]

#@markdown ---
#@markdown ### Generation Quality
INFERENCE_STEPS = 25 #@param {type:"slider", min:15, max:50, step:5}
GUIDANCE_SCALE = 7.5 #@param {type:"slider", min:5, max:12, step:0.5}

#@markdown ---

print("📋 Configuration:")
print(f"   Story: {STORY_PROMPT[:60]}...")
print(f"   Style: {STYLE}")
print(f"   Layout: {LAYOUT}")
print(f"   Steps: {INFERENCE_STEPS}")
print(f"   Guidance: {GUIDANCE_SCALE}")

## 📝 Cell 4: Generate Scene Plan

Uses Gemini 2.0 Flash to convert your story prompt into a structured scene plan.

In [ ]:
#@title 📝 Generate Scene Plan with Gemini

import json
import os

os.chdir('/content/manga-gen')

# Run scene generator
!python scripts/generate_scene_json.py "{STORY_PROMPT}" --style {STYLE} --layout {LAYOUT} --output scene_plan.json

# Display result
if os.path.exists('scene_plan.json'):
    with open('scene_plan.json', 'r') as f:
        scene = json.load(f)
    
    print("\n" + "="*50)
    print("📋 Scene Plan Generated!")
    print("="*50)
    print(f"Title: {scene.get('title', 'Untitled')}")
    print(f"Panels: {len(scene.get('panels', []))}")
    print(f"Characters: {[c['name'] for c in scene.get('characters', [])]}")
else:
    print("❌ Scene generation failed. Check the output above.")

## 🎨 Cell 5: Generate Panel Images (GPU)

This is the main step! Uses:
- **SDXL** for high-quality anime/manga images
- **IP-Adapter** for character consistency (if available)

**First run:** ~5-8 minutes (downloads models to Drive)
**Subsequent runs:** ~3-5 minutes (loads from cache)

In [ ]:
#@title 🎨 Generate Panel Images
#@markdown This will take 3-8 minutes depending on cache status.

import torch
import time
import os

os.chdir('/content/manga-gen')
os.makedirs('outputs', exist_ok=True)

print("🎨 Starting image generation...")
print(f"   Using cache: {os.environ.get('HF_HOME', 'default')}")
print("")

start_time = time.time()

# Run panel generator
!python scripts/generate_panels.py \
    --scene scene_plan.json \
    --output outputs/ \
    --steps {INFERENCE_STEPS} \
    --guidance {GUIDANCE_SCALE}

elapsed = time.time() - start_time
print(f"\n⏱️ Total time: {elapsed/60:.1f} minutes")

# Check if real images were generated (not mock)
if elapsed < 60:  # Less than 1 minute = probably mock
    print("\n⚠️ Generation was very fast - might be mock mode.")
    print("   Check if there were any errors above.")
else:
    print("\n✅ Real images generated!")

In [ ]:
#@title 🖼️ Display Generated Panels

from IPython.display import display, Image as IPImage
import glob
import os

os.chdir('/content/manga-gen')

print("\n🖼️ Generated Panels:")
print("="*50)

panels = sorted(glob.glob('outputs/panel_*.png'))
panels = [p for p in panels if 'with_bubbles' not in p]

if panels:
    for panel in panels:
        print(f"\n{os.path.basename(panel)}")
        display(IPImage(filename=panel, width=400))
else:
    print("❌ No panels found. Check generation output above.")

# Character refs
refs = glob.glob('outputs/character_refs/*.png')
if refs:
    print("\n📸 Character References:")
    for ref in refs:
        print(f"\n{os.path.basename(ref)}")
        display(IPImage(filename=ref, width=200))

## 💬 Cell 6: Place Dialogue Bubbles

In [ ]:
#@title 💬 Place Dialogue Bubbles

import os
os.chdir('/content/manga-gen')

!python scripts/place_bubbles.py \
    --panels outputs/ \
    --scene scene_plan.json \
    --output bubbles.json

# Display bubble info
import json
if os.path.exists('bubbles.json'):
    with open('bubbles.json', 'r') as f:
        bubbles = json.load(f)
    print("\n💬 Bubbles placed:")
    for panel, bs in bubbles.items():
        print(f"   {panel}: {len(bs)} bubble(s)")

## 📄 Cell 7: Compose Final Page

In [ ]:
#@title 📄 Compose Final Page & PDF

import os
os.chdir('/content/manga-gen')

!python scripts/compose_page.py \
    --panels outputs/ \
    --bubbles bubbles.json \
    --scene scene_plan.json \
    --output outputs/

# Display final page
if os.path.exists('outputs/manga_page.png'):
    print("\n🎨 Final Manga Page:")
    from IPython.display import display, Image as IPImage
    display(IPImage(filename='outputs/manga_page.png', width=600))

## 📥 Cell 8: Download Your Manga!

In [ ]:
#@title 📥 Download Options

import os
import shutil
from google.colab import files

os.chdir('/content/manga-gen')

print("\n" + "="*50)
print("🎉 YOUR MANGA IS READY!")
print("="*50)

# Check files
zip_path = 'manga_output.zip'
pdf_path = 'outputs/manga_page.pdf'
png_path = 'outputs/manga_page.png'

# File info
print("\n📁 Output Files:")
for f in [zip_path, pdf_path, png_path]:
    if os.path.exists(f):
        size = os.path.getsize(f) / 1024 / 1024
        print(f"   ✅ {f} ({size:.2f} MB)")
    else:
        print(f"   ❌ {f} not found")

# Copy to Drive if mounted
drive_output = '/content/drive/MyDrive/MangaGen_Outputs'
if os.path.exists('/content/drive'):
    os.makedirs(drive_output, exist_ok=True)
    if os.path.exists(zip_path):
        shutil.copy(zip_path, drive_output)
    if os.path.exists(pdf_path):
        shutil.copy(pdf_path, drive_output)
    print(f"\n📂 Copied to Google Drive: {drive_output}")

print("\n📥 Download options:")
print("   1. Check your Google Drive: MangaGen_Outputs/")
print("   2. Click the download button below:")

# Download button
if os.path.exists(zip_path):
    files.download(zip_path)

---

## 🎁 Bonus: Generate Another Page

Want to generate more panels? Just update the story prompt and run cells 4-8 again!

In [ ]:
#@title 🔄 Quick Regenerate (Same Settings)
#@markdown Run this to regenerate with the same settings.

import os
os.chdir('/content/manga-gen')

# Clear previous outputs
!rm -rf outputs/*

print("🔄 Regenerating with same settings...")
print("")

# Run full pipeline
!python scripts/generate_panels.py --scene scene_plan.json --output outputs/ --steps {INFERENCE_STEPS}
!python scripts/place_bubbles.py --panels outputs/ --scene scene_plan.json
!python scripts/compose_page.py --panels outputs/ --bubbles bubbles.json --scene scene_plan.json

# Display
from IPython.display import display, Image as IPImage
if os.path.exists('outputs/manga_page.png'):
    print("\n🎨 New Page:")
    display(IPImage(filename='outputs/manga_page.png', width=600))

---

## 📊 Pipeline Summary

| Step | Component | First Run | Cached |
|------|-----------|-----------|--------|
| 1 | Setup & Install | ~3 min | ~30 sec |
| 2 | Scene Plan (Gemini) | ~5 sec | ~5 sec |
| 3 | Panel Generation (SDXL) | ~8 min | ~4 min |
| 4 | Bubble Placement | ~10 sec | ~10 sec |
| 5 | PDF Composition | ~5 sec | ~5 sec |

**Total:** ~12 min first run, ~5 min cached

---

Made with ❤️ by Barun | [GitHub](https://github.com/Barun-2005/manga-gen-ai-pipeline)